In [1]:
print("\n\t This ML programme, predicts the room-rates of hotels")



	 This ML programme, predicts the room-rates of hotels


In [ ]:
train_data_path = "./data/processed_data.csv"
import pandas as pd
processed_data = pd.read_csv(train_data_path)
# processed_data


,Room Rate,Month Record,Number of Star,Hotel Location,Number of Nights,Room-type Name
0,0.0,10,3,Residence area,1.0,SUPERIOR
1,47500.0,10,3,Residence area,2.0,SUPERIOR
2,95000.0,11,3,Residence area,2.0,DELUXE
3,60000.0,11,3,Residence area,1.0,SUPERIOR
4,100000.0,11,3,Residence area,1.0,SUITE
...,...,...,...,...,...,...
5583,40000.0,1,4,Commercial area,2.0,DELUXE
5584,40000.0,1,4,Commercial area,2.0,DELUXE
5585,40000.0,1,4,Commercial area,1.0,SUPER CLASSIC
5586,40000.0,1,4,Commercial area,1.0,DELUXE


In [128]:
# drop rowws whose room-rates are zero
required_data = processed_data[processed_data['Room Rate'] > 0]

# Implement one-hot encoding for Hotel-Location
hotel_location = required_data['Hotel Location']
new_location_encoding = hotel_location.replace(to_replace=["Residence area", "Commercial area"], value=[1, 2])
required_data["Hotel Location"] = new_location_encoding

# Implement one-hot encoding for Room-type name
room_type_val = required_data['Room-type Name']
new_room_type_encoding = room_type_val.replace(to_replace=[
    "DELUXE", "DELUXE ROOM", "DELUX ROOM", "DELUXE SUITE", "CLASSIC KING", "CLASSIC ROOM", "CLASSIC SUITE", 
    "DIPLOMATIC", "DIPLOMATIC SUITE", 
    "SUITE", "SUPERIOR", "SUPERIOR STUDIO", "SUPERIOR ROOM", 
    "EXECUTIVE SUITE", "EXECUTIVE ROOM", 
    "SUPER CLASSIC", "DIPLOMATIC FLAT", 
    "1 BEDROOM APARTMENT", "STANDARD", "MINI FLAT", "CLASSIC STUDIO", 
    "2 BEDROOM APARTMENT", "MAINTENANCE BLOCK", "PRESIDENTIAL SUITE", 
    'DUPLEX(ROYAL SUITE)', 'PLATINUM STUDIO SUITE', 'GLUTTO TWIN APARTMENT', 
    'JUNIOR SUITE', 'APARTMENT', 'GLUTTO ROOM', 'GLUTTO SUITE'
    ], value=[1, 1, 1, 1, 2, 2, 2, 3, 3, 4, 5, 5, 5, 6, 6, 7,8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22])
required_data["Room-type Name"] = new_room_type_encoding
required_data


C:\Users\HP\AppData\Local\Temp\ipykernel_6880\3423311706.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  new_location_encoding = hotel_location.replace(to_replace=["Residence area", "Commercial area"], value=[1, 2])
C:\Users\HP\AppData\Local\Temp\ipykernel_6880\3423311706.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  required_data["Hotel Location"] = new_location_encoding
C:\Users\HP\AppData\Local\Temp\ipykernel_6880\3423311706.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be 

,Room Rate,Month Record,Number of Star,Hotel Location,Number of Nights,Room-type Name
1,47500.0,10,3,1,2.0,5
2,95000.0,11,3,1,2.0,1
3,60000.0,11,3,1,1.0,5
4,100000.0,11,3,1,1.0,4
5,46875.0,11,3,1,1.0,5
...,...,...,...,...,...,...
5583,40000.0,1,4,2,2.0,1
5584,40000.0,1,4,2,2.0,1
5585,40000.0,1,4,2,1.0,7
5586,40000.0,1,4,2,1.0,1


In [129]:
# Select targets and features
target = required_data["Room Rate"]
# Select feature
features = ["Month Record", "Number of Star", "Hotel Location", "Number of Nights", "Room-type Name"]
features = required_data[features]

In [130]:
from sklearn.model_selection import train_test_split #
# the random_state argument ensures the same split is given always 
target_values = target.values
feature_values = features.values
train_X, val_X, train_y, val_y = train_test_split(feature_values, target_values, random_state=0)
train_X

array([[ 6.,  4.,  2.,  2.,  1.],
       [11.,  4.,  2.,  1.,  1.],
       [ 9.,  3.,  1.,  1.,  5.],
       ...,
       [ 7.,  3.,  1.,  1.,  5.],
       [10.,  4.,  1.,  1.,  2.],
       [11.,  4.,  1.,  1.,  2.]])

In [131]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

model = RandomForestRegressor(random_state=1)
model.fit(train_X, train_y)

RandomForestRegressor(random_state=1)